In [16]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Input, Concatenate, Conv2D, Flatten, Dense, Embedding, LSTM
from keras.models import Model

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re



## Read data using pandas


In [17]:
path1 = "/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe21]-9. Hausaufgabe - Pflichttest C-Antworten_plagiate.csv"
path2 = "/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe21]-9. Hausaufgabe - Pflichttest C-Antworten_labled.csv"
path3 = "/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [WS2021]-9. Hausaufgabe - Pflichttest C-Antworten_labled.csv"
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)
df3 = df3.drop('Unnamed: 0', axis=1)

This is what the data looks like:


In [18]:
# df1.head(2)
# df2.head(2)
# df2.dtypes

## Remove Given Code from Student Solution


In [19]:
from re import T
from bs4 import BeautifulSoup

def get_given_code(file):
    try:
        with open(file) as xmlstr:
            soup = BeautifulSoup(xmlstr, 'xml')
            answerpreload = soup.find('answerpreload').text
            questiontext = soup.find('questiontext').text
            return answerpreload, questiontext
    except FileNotFoundError:
        return "Keine Vorgabedatei im Repo gefunden", "Keine Vorgabedatei im Repo gefunden"


def remove_given_code(code, preload_file_path):
    answerpreload, _ = get_given_code(preload_file_path)
    # remove empty lines
    code = '\n'.join([s for s in code.splitlines() if s.strip() != ''])
    if answerpreload == 'Keine Vorgabedatei im Repo gefunden':
        return code
    # print(answerpreload)
    answerpreload = answerpreload.replace('\t', '').replace('\r', '')
    answerpreload = '\n'.join([s for s in answerpreload.splitlines() if s.strip() != ''])
    for ap_tmp in answerpreload.splitlines():
        ap_tmp = re.escape(ap_tmp)
        ap_tmp = '^' + ap_tmp + '$'
        # {{ cr_random.f1 }} --> \S*
        ap_tmp = re.sub(
            r"\\{\\{\\\s*\S+\s*\\}\\}", r"\\S*", ap_tmp)
        for code_tmp in code.splitlines():
            # print((re.match(ap_tmp, code_tmp.replace('\t', '').replace('\r', ''))!=None, ap_tmp, code_tmp))
            if re.match(ap_tmp, code_tmp.replace('\t', '').replace('\r', '')):
                code = code.replace(code_tmp+'\n', '', 1)
                break
    return code


In [20]:
df_list = [df1, df2]

def remove_given_code_from_df(df_list):
    for df in df_list:
        semester = df["semester"].values[0]
        ha = df["ha"].values[0]
        prog_lang = df["prog_lang"].values[0]
        task = df["task"].values[0]
        answerpreload_path = f'../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_lang}-Antworten_{task}.xml'
        df['code1'] = [remove_given_code(code, answerpreload_path) for code in df['code1']]
        df['code2'] = [remove_given_code(code, answerpreload_path) for code in df['code2']]

remove_given_code_from_df(df_list)


## Use WS2021 Dataset


In [21]:
# prüfe ob code von Aufgabe a oder b ist und entferne die jeweilige vorgabe
def remove_given_code_from_df_WS2021(df_list):
    for df in df_list:
        semester = df["semester"].values[0]
        ha = df["ha"].values[0]
        prog_lang = df["prog_lang"].values[0]
        task = df["task"].values[0]
        number = int(re.findall('(\d+)', task)[0])
        task_2 = re.sub('\d+', str(number-1), task)
        answerpreload_path = f'../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_lang}-Antworten_{task}.xml'
        alternative_answerpreload_path = f'../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_lang}-Antworten_{task_2}.xml'
        # möglich das mehr automatisiert zu machen?
        ap = re.escape('char test[11]= "0123456789";')
        df['code1'] = [remove_given_code(code, alternative_answerpreload_path) if re.findall(ap,code) else remove_given_code(code, answerpreload_path) for code in df['code1']]
        df['code2'] = [remove_given_code(code, alternative_answerpreload_path) if re.findall(ap,code) else remove_given_code(code, answerpreload_path) for code in df['code2']]

remove_given_code_from_df_WS2021([df3])


Concat the DataFrames


In [22]:
df_list = [df1, df2]
concat_df = pd.concat(df_list, ignore_index=True)
concat_df = concat_df.drop('Unnamed: 0', axis=1)

## Split train and test data


In [23]:
X_temp, X_test, y_temp, y_test = train_test_split(concat_df[['code1', 'code2']], concat_df['label'], test_size=0.2, random_state=42)

In [24]:
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

In [25]:
#TODO remove newlines and ect
def striphtml(data): 
    return re.sub('<.*?>', ' ', str(data)) 

def stripunc(data): 
    return re.sub('[^A-Za-z%\._\[\]]+', ' ', str(data), flags=re.MULTILINE|re.DOTALL) 

In [26]:
# csv_path = "/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [WS2021]-9. Hausaufgabe - Pflichttest C-Antworten_labled.csv"
# df_test_remove = pd.read_csv(csv_path)

# semester = 'WS2021'
# ha = '9'
# prog_language = 'C'
# task = 'Antwort 9'
# answerpreload_path = f'../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml'
# # print(df_test_remove['code2'].values[2])
# result = remove_given_code(df_test_remove['code2'].values[2], answerpreload_path)
# # print(result)


## Preprocess the pairs for the models


In [27]:
def preprocess_student_pairs(data, tokenizer): 
    # answerpreload_path = f'../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml'
    # data['code1_strip'] = data[['code1']].apply(lambda x:remove_given_code(x[0]), axis=1)
    # data['code2_strip'] = data[['code2']].apply(lambda x:striphtml(stripunc((x[0]))), axis=1)
    data['code1_strip'] = data[['code1']].apply(lambda x:striphtml(stripunc((x[0]))), axis=1)
    data['code2_strip'] = data[['code2']].apply(lambda x:striphtml(stripunc((x[0]))), axis=1)
    data['code12'] = data[['code1_strip','code2_strip']].apply(lambda x:str(x[0])+" "+str(x[1]), axis=1)
    tokenizer.fit_on_texts(data['code12'].values)
    data['code1_tokend'] = data['code1_strip'].apply(lambda x:tokenizer.texts_to_sequences([str(x)]))
    data['code2_tokend'] = data['code2_strip'].apply(lambda x:tokenizer.texts_to_sequences([str(x)]))
    data['code1_padded'] = data['code1_tokend'].apply(lambda x:pad_sequences(x, maxlen=256, padding='post', truncating='post'))
    data['code2_padded'] = data['code2_tokend'].apply(lambda x:pad_sequences(x, maxlen=256, padding='post', truncating='post'))
    import numpy as np
    data['code12_padded'] = [np.concatenate((x[0], x[1]), axis=None) for x in data[['code1_padded','code2_padded']].values]
    x = [np.asarray(x).astype('int32') for x in data['code12_padded']]
    x = tf.convert_to_tensor(x)
    # x = tf.convert_to_tensor(x)
    x_array = np.array(x)
    return x_array, tokenizer

In [13]:
#TODO fit_on_text nur für x_train benutzen
#TODO tokenizer einmal machen und dann speichern und laden
#TODO anderen algo benutzen
tokenizer = Tokenizer()
x_val,tokenizer = preprocess_student_pairs(X_val,tokenizer)
x_train,tokenizer = preprocess_student_pairs(X_train,tokenizer)
x_test,tokenizer = preprocess_student_pairs(X_test,tokenizer)
X_test_df3 = preprocess_student_pairs()


2022-10-29 14:09:07.653740: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-29 14:09:07.654012: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-29 14:09:07.654025: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (erik-MS-7C56): /proc/driver/nvidia/version does not exist
2022-10-29 14:09:07.655130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### My Model


In [53]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(256*2,), dtype='int32')),
model.add(tf.keras.layers.Dense(256*2, activation='relu')),
model.add(tf.keras.layers.Dense(256*2, activation='relu')),
model.add(tf.keras.layers.Dense(256*2, activation='relu')),
# model.add(tf.keras.layers.Dense(4, activation='relu')),
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.build(input_shape=(1,256))
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)



In [54]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(189, 512)
(189,)
(48, 512)
(48,)


In [55]:
model.fit(x_train, y_train.values.reshape(-1,1), batch_size=32, epochs=5, validation_data=(x_val,y_val.values.reshape(-1,1)))


Epoch 1/5
6/6 [==============================] - 0s 20ms/step - loss: 1.2924 - accuracy: 0.8307 - val_loss: 3.2557 - val_accuracy: 0.8333
Epoch 2/5
6/6 [==============================] - 0s 6ms/step - loss: 1.9831 - accuracy: 0.8995 - val_loss: 3.4385 - val_accuracy: 0.9375
Epoch 3/5
6/6 [==============================] - 0s 6ms/step - loss: 0.3927 - accuracy: 0.9683 - val_loss: 0.6897 - val_accuracy: 0.9583
Epoch 4/5
6/6 [==============================] - 0s 6ms/step - loss: 0.2776 - accuracy: 0.9471 - val_loss: 0.3575 - val_accuracy: 0.9792
Epoch 5/5
6/6 [==============================] - 0s 6ms/step - loss: 0.1351 - accuracy: 0.9683 - val_loss: 0.9293 - val_accuracy: 0.9792


In [ ]:
print(len(X_train['code12_padded']))
print(len(X_train['code12_padded'][0]))
print(X_train['code12_padded'].dtype)
print(X_train['code12_padded'][0].dtype)

# def show_shapes(): # can make yours to take inputs; this'll use local variable values
#     print("Expected: (num_samples, timesteps, channels)")
#     print("Sequences: {}".format(Sequences.shape))
#     print("Targets:   {}".format(Targets.shape))

[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
Total params: 788,481
Trainable params: 788,481
Non-trainable params: 0
_________________________________________________________________


### Evaluate my Model


In [58]:
model.evaluate(x_test, y_test.values)
model.evaluate(x_val, y_val.values)


2/2 [==============================] - 0s 2ms/step - loss: 0.3189 - accuracy: 0.9844


[0.31894052028656006, 0.984375]

In [45]:
# with open('model_summary.txt', mode='w') as file:
#     model.summary(print_fn=lambda x: file.write(x + '\n'))
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
# numeric_dataset = tf.data.Dataset.from_tensor_slices((new_df, label))

# predict = model.predict(new_df[9000:15000])

# evaluate 64_hand_labled_pairs
# plt.show()

# print(label)
# zero = 0
# one = 0
# for row in label.values[:9000]:
#     if row == 0:
#         zero = zero +1
#     elif row == 1:
#         one = one +1
# print(f'training_data: zeros = {zero}; ones = {one}')
# zero = 0
# one = 0
# for row in label.values[9000:10000]:
#     if row == 0:
#         zero = zero +1
#     elif row == 1:
#         one = one +1
# print(f'test_data: zeros = {zero}; ones = {one}')


188/188 [==============================] - 0s 861us/step


## My first Baseline Model


## Simple Text Classification


In [ ]:
#https://www.tensorflow.org/tutorials/keras/text_classification
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])


### Quora siames model


In [41]:
from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import (
#     BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
# )
from keras.layers.pooling import MaxPooling2D
from tensorflow.keras.layers import concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
from keras import backend as K

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


In [ ]:
not_present_list = []
vocab_size = len(t.word_index) + 1
print('Loaded %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((vocab_size, len(embeddings_index['no'])))
for word, i in t.word_index.items():
    if word in embeddings_index.keys():
        embedding_vector = embeddings_index.get(word)
    else:
        not_present_list.append(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.zeros(300)


In [49]:
# x_array
len(y_train.values)

189

In [50]:
input_1 = Input(shape=(x_array.shape[1],))
input_2 = Input(shape=(x_array.shape[1],))


common_embed = Embedding(name="synopsis_embedd",input_dim =len(t.word_index)+1, 
                       output_dim=len(embeddings_index['no']),weights=[embedding_matrix], 
                       input_length=training_text1.shape[1],trainable=False) 
lstm_1 = common_embed(input_1)
lstm_2 = common_embed(input_2)


common_lstm = LSTM(64,return_sequences=True, activation="relu")
vector_1 = common_lstm(lstm_1)
vector_1 = Flatten()(vector_1)

vector_2 = common_lstm(lstm_2)
vector_2 = Flatten()(vector_2)


NameError: name 'Input' is not defined

In [ ]:

x3 = Subtract()([vector_1, vector_2])
x3 = Multiply()([x3, x3])

x1_ = Multiply()([vector_1, vector_1])
x2_ = Multiply()([vector_2, vector_2])
x4 = Subtract()([x1_, x2_])
    
    #https://stackoverflow.com/a/51003359/10650182
x5 = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([vector_1, vector_2])
    
conc = Concatenate(axis=-1)([x5,x4, x3])

x = Dense(100, activation="relu", name='conc_layer')(conc)
x = Dropout(0.01)(x)
out = Dense(1, activation="sigmoid", name = 'out')(x)

quora_model = Model([input_1, input_2], out)

quora_model.compile(loss="binary_crossentropy", metrics=['acc',auroc], optimizer=Adam(0.00001))

### Reproduce the binary accuracy


In [ ]:
# tf.keras.metrics.BinaryAccuracy()
m = tf.keras.metrics.BinaryAccuracy()
sum = 0
zeros = 0
ones = 0
for p,l in zip(predict, label[9000:15000]):
    m.update_state([[l]],[[p]])
    print(f"p: {p}; l: {l}; accuracy: {m.result().numpy()}")
    if m.result().numpy():
        sum = sum + 1
        if l:
            ones = ones + 1
        else:
            zeros = zeros + 1
    m.reset_state()
print(f"total accuracy: {sum/(15000-9000)}")
print(f"number positive accuracy through zeros (round off): {zeros}")
print(f"number positive accuracy through ones (round up): {ones}")

## InifiniteMonkey


In [ ]:
# def build_infinitemonkey_baseline():
    # n grams
    # tf-idf weighting
    # cosine simularity
    # grid search


# def build_infinitemonkey_model():
    # 2x encoder
    # embedding (32)
    # BatchNorm
    # LSTM(128)
    # 2x comparison module
    # Concat the output of the two encoders as input
    # BatchNorm
    # Dense (128)
    # ReLu
    # 1x network
    # Sum the output of the two comparison moduels as input
    # BatchNorm
    # Dense (128)
    # ReLu
    # BatchNorm
    # Dense(1)
    # Sigmoid


### Plot the predictions


In [38]:
# print(predict)
%matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize = (10,5))
plt.plot(predict)
# plt.savefig('predictions_of_test_data.jpg')
# step_size = np.arange(0.05,0.95,0.05)
# step_size[1]
# # plt.bar(step_size, np.round(predict,2))
# for i in range(0,10):
#     np.round(predict,2)

Using matplotlib backend: TkAgg


<Figure size 720x360 with 0 Axes>

### Save/Load our trained model


In [59]:
import os.path
path_to_model = '../data/model/new_trained_simple_model'
if os.path.isfile(path_to_model) is False:
    model.save(path_to_model)
# from tensorflow.keras.models import load_model
# model = load_model(path_to_model)

INFO:tensorflow:Assets written to: ../data/model/new_trained_simple_model/assets
